# Spark und RDDs
Apache Spark ist ein leistungsfähiges Open-Source-Framework für die Verarbeitung großer Datenmengen auf verteilten Systemen. Es ist für seine Vielseitigkeit - insbesondere im Vergleich zu MapReduce - und die Verwendung von Resilient Distributed Datasets (RDDs) bekannt.

In [ ]:
from pyspark import SparkContext

from tui_dsmt.parallel.datasets import text_paths

## Inhaltsverzeichnis
- [Apache Spark](#Apache-Spark)
- [Resilient Distributed Datasets](#Resilient-Distributed-Datasets)
- [Apache Spark und Python](#Apache-Spark-und-Python)
- [Zusammenfassung](#Zusammenfassung)

## Apache Spark
Apache Spark ist ein Open-Source-System zur verteilten Verarbeitung großer Datenmengen. Die erste Version wurde 2014 veröffentlicht (und erst später an die Apache Software Foundation gespendet) und sollte Limitierungen älterer Verfahren überwinden. Der Ablauf von MapReduce ist beispielsweise sehr statisch: Daten werden (verteilt) gelesen, eine Map-Funktion wird auf sie angewandt, anschließend werden die aus der Map-Funktion resultierenden Schlüssel-Wert-Paare reduziert und wieder abgespeichert. MapReduce eignet sich daher ausschließlich für die Batch-Verarbeitung von Daten. Für Algorithmen, die zwangsweise mehrere Iterationen benötigen, wie das beispielsweise beim k-Means Verfahren notwendig ist, ist MapReduce ebenfalls wenig geeignet.

Für Spark wurden daher einige grundsätzliche Designentscheidungen getroffen:
- Die Verarbeitung kann In-Memory, also ohne Persistierung der Ergebnisse, stattfinden.
- Es werden sowohl Batch- als auch Streaming-Daten unterstützt.
- Eine Vielzahl von Programmiersprachen, darunter Python, können verwendet werden.
- Umfangreiche Bibliotheken, z.B. für HDFS, SQL oder maschinelles Lernen, sollen die Benutzerfreundlichkeit erhöhen.
- Resilient Distributed Datasets dienen als darunterliegende Datenstruktur, um Verteilbarkeit und Fehlertoleranz sicherzustellen.

## Resilient Distributed Datasets
Resilient Distributed Datasets (RDDs) sind die zentrale Datenabstraktion in Apache Spark. Sie eröffnen die Möglichkeit einer einfachen, aber verteilten Ausführung verschiedener Transformations- und Aktionsoperationen auf großen Datenmengen. RDDs haben dazu einige Eigenschaften:
1. RDDs sind in **Partitionen** unterteilt, die auf verschiedenen Systemen eines Clusters gespeichert werden, um Parallelität zu ermöglichen.
2. **Transformationen** sind Operationen wie `map` oder `filter`, verändern also die Daten eines RDD. Transformationen werden grundsätzlich *lazy* angewandt (*lazy evaluation*).
3. **Aktionen** sind Operationen wie `collect` oder `count`. Sie lösen tatsächlich Berechnungen aus und geben Ergebnisse zurück.
4. RDDs sind **unveränderlich** (immutable). Jede Transformation eines RDD erzeugt ein neues, davon abgeleitetes RDD.
5. Die Verknüpfungen von RDDs sind durch einen **gerichteten, azyklischen Graph** darstellbar.

Die Vorteile der RDDs lassen sich somit sehr einfach zusammenfassen:
- Durch die Partitionierung der RDDs liegen diese verteilt vor.
- Durch Spark implementierte Transformationen und Aktionen nutzen diese Parallelität aus.
- Ist ein RDD durch einen Ausfall eines Teilsystems nicht (mehr) verfügbar, kann es automatisch wiederhergestellt werden, indem die erforderlichen Teilschritte über den Abhängigkeitsgraphen gesucht und (erneut) ausgeführt werden.

In Spark 2.x steht neben der RDD API auch die Dataset API zur Verfügung. Sie bietet einen High-Level-Zugriff auf die Daten, welcher leichter durch die Spark Engine optimiert werden kann. Auch wenn die Verwendung der Dataset API nahegelegt wird, gilt die RDD API dennoch noch nicht als veraltet.

## Apache Spark und Python
PySpark bietet die Leistungsfähigkeit von Apache Spark kombiniert mit der Einfachheit und Flexibilität von Python. Es unterstützt viele Spark-Features wie die Integration von SQL, eine Anbindung an Pandas und Machine Learning. Die Verwendung eines objektorientierten Programmiermodells bildet dabei die Beziehung der RDDs ab und erlaubt *lazy evaluation*. Auf Grund der [Anzahl an Operationen](https://spark.apache.org/docs/latest/api/python/index.html) sollen an dieser Stelle nur wenige dieser anhand des Word-Count Beispiel aufgegriffen und die generelle Verwendung von PySpark gezeigt werden.

In [ ]:
# Das Erstellen eines Spark-Kontexts ist per Kontext-
# Manager möglich. 'local' gibt an, dass kein Cluster,
# sondern ein einzelner Knoten zur Ausführung
# verwendet werden soll.
with SparkContext('local', 'word count') as sc:
    # Lokale Dateien können mit Hilfe ihres Pfads
    # geladen werden.
    input_rdd = sc.textFile(','.join(text_paths))

    # Ein Aufruf von collect "sammelt" das aktuelle
    # Ergebnis. Beim textFile wäre das eine Liste
    # bestehend aus den einzelnen Zeilen *aller*
    # bereitgestellten Dateien.
    #all_lines = input_rdd.collect()
    #print(ww)

    # Map wendet eine Funktion auf jedes Element
    # einer Liste an und gibt damit eine Liste
    # der selben Größe zurück. flatMap
    # funktioniert analog, gibt aber eine flache
    # Liste zurück, sodass jedes Element auf
    # beliebig viele abgebildet werden und die
    # Ergebnisliste damit eine abweichende
    # Länge besitzen kann.
    words_rdd = input_rdd.flatMap(lambda line: line.split(' '))

    # Durch Split können leere Strings entstehen,
    # die zunächst gefiltert werden.
    filtered_words_rdd = words_rdd.filter(lambda word: word)

    # Wie bei MapReduce werden die einzelnen
    # Wörter jetzt auf Paare der Form
    # (Wort, 1) abgebildet.
    word_tuples_rdd = filtered_words_rdd.map(lambda word: (word, 1))

    # Die Reduktionsfunktion kann verwendet werden,
    # um Werte (hier immer 1) anhand ihres
    # Schlüssels zusammenzufassen und anschließend
    # zu aggregieren.
    word_counts_rdd = word_tuples_rdd.reduceByKey(lambda a, b: a + b)

    # Zuletzt wird das Ergebnis des finalen RDD
    # angefordert. Erst damit wird die Berechnung
    # ausgelöst. Auskommentieren der nachfolgenden
    # Zeile führt dazu, dass keinerlei Berechnung
    # vorgenommen wird.
    word_counts = word_counts_rdd.collect()

for word, count in word_counts[:10]:
    print(word, count)

Anmerkungen zum Beispiel:
- Es ergibt sich ein linearer Ausführungsgraph. Durch die mehrfache Verwendung eines RDD könnten allerdings Zwischenergebnisse wiederverwendet werden und ein komplexerer Ausführungsgraph würde entstehen.
- Allein die Verwendung der Spark-Funktionen reicht aus, um parallel zu rechnen: RDDs werden automatisch im Cluster partitioniert und Funktionen implizit verteilt berechnet.
- Die Erzeugung nicht verwendeter RDDs ergibt keinen Performance-Nachteil, da deren Ergebnisse gar nicht erst berechnet werden.
- Durch Anpassung des Kontexts lassen sich derartige Berechnungen lokal testen und anschließend relativ einfach auf einen Cluster übertragen.

## Zusammenfassung
Spark ist ein vielseitiges und effizientes Framework zur verteilten Berechnung. Das Konzept der RDDs erlaubt dabei eine verständliche und abstrakte Programmierung, während Implementierungen für verschiedene Programmiersprachen zur Verfügung stehen.